### Tutorial 1: Adding a new monatomic species entry to your local data store, the manual way

If you would like to do calculations using a plasma species that you haven't used before, you will need to generate a minplascalc data entry for it first. You only need to do this <i>once</i> - minplascalc will store the species data in a plain-text file formatted using JSON syntax, and it will then be available for use in any of your future calculations.

For monatomic species minplascalc has built-in readers for handling data obtained from the energy levels section of NIST Atomic Spectra Database, which can be found at http://physics.nist.gov/PhysRefData/ASD/levels_form.html. The landing page looks like this.

<img src="demoimg/asd_landingpage.png" alt="NIST ASD landing page" style="width: 800px;"/>

You must then specify the atom or ion that you want to retrieve energy level information for. Let's get the data for the singly-charged oxygen cation species O<sup>+</sup>, which is "O II" in spectrographic terminology. Enter the identifier in the Spectrum field on the form:

<img src="demoimg/asd_spectrumidentification.png" alt="Spectrum identification" style="width: 400px;"/>

Make sure you set Level Units to "cm-1", Format Output to "ASCII (text)", and uncheck everything in the output section except Level and J, like this:

<img src="demoimg/asd_retrievalsettings.png" alt="Spectrum identification" style="width: 400px;"/>

Click the Retrieve Data button. You should see a page with the energy levels listed in a plain-text table. Click and drag to select everything from the first energy level line down to the last one before the first ionisation energy entry (these have no J value and are separated in boxes of ASCII dashes). While you're there, make a note of the ionisation energy value.

<table style="border:none!important">
<td><img src="demoimg/asd_leveldata1.png" alt="Spectrum identification" style="height: 400px;"/>
<td><img src="demoimg/asd_leveldata2.png" alt="Spectrum identification" style="height: 400px;"/>
<td><img src="demoimg/asd_leveldata3.png" alt="Spectrum identification" style="height: 400px;"/>
</table>

Copy and paste the selected content into a temporary text file. This has already been done for the oxygen cation, and the raw NIST data file is located at minplascalc/notebooks/demodata/nist_O+. Running the following code snippet will create a minplascalc Species object for the O<sup>+</sup> ion and then store it to a file for later reuse:

In [1]:
import minplascalc as mpc
from scipy import constants

invcm_to_joule = constants.Boltzmann / (0.01*constants.value('Boltzmann constant in inverse meters per kelvin'))

energylevels = mpc.readers.nist_energylevels(open('./demodata/nist_O+'))
for el in energylevels:
    el[1] *= invcm_to_joule

oxygenplus = mpc.MonatomicSpecies(
    name='O+',
    stoichiometry={'O': 1},
    molarmass=0.0159994,
    chargenumber=1,
    ionisationenergy=invcm_to_joule*283270.9,
    energylevels=energylevels,
    sources=['NIST Atomic Spectra Database (ver. 5.3), [Online]. A Kramida, '
             'Yu Ralchenko, J Reader, and NIST ASD Team, National Institute '
             'of Standards and Technology, Gaithersburg MD., '
             'http://physics.nist.gov/asd'])

mpc.species_to_file(oxygenplus)

What's happening here? First we import the minplascalc package, then we read the energy levels from the NIST data file (converting to J), then we create a `MonatomicSpecies` object, and finally we save the contents to a data file.

The `MonatomicSpecies` class constructor takes as arguments the species name, a small dictionary describing the elemental stoichiometry of the species (in this case a single oxygen atom), the molar mass in kg/mol, the charge on the species in units of the fundamental charge (in this case 1 because O<sup>+</sup> is singly charged), the ionisation energy of the species in J, a list of length-2 lists describing the energy levels (degeneracy and energy in J of each level respectively), and a list of data sources for referencing. The object can be written out to disk using the `species_to_file` utility function, which saves data to a JSON-formatted file. It takes two arguments: a minplascalc Species object, and an optional filename - if omitted, the filename is created using the species' name.

After this process it will be possible to create an O<sup>+</sup> species object in any minplascalc calculation by importing it using either the explicit path to the JSON file, or (preferably) just the name of the species provided the JSON file is stored in any of the standard minplascalc data paths - see later tutorials for examples.